In [1]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [59]:
querys={
    
    'TargetEstatistico':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatistico
    WHERE [Tipo de Operação]<>'OUTROS' AND Situação='FATURADO'
    AND MONTH([Data de Faturamento])=MONTH(GETDATE())    
    
    """,
    
    'Produto':
    
    """
    
    SELECT SKU,[Cód. Fabricante],Produto,Status,Fabricante,Departamento,Seção,Categoria,Linha FROM netfeira.vw_produto
    
    """
    
}

In [60]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [61]:
tabelas_df['TargetEstatistico'].columns

Index(['Origem', 'ID Empresa', 'ID Cliente', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV', 'Peso Bruto KG',
       'Peso Líquido KG'],
      dtype='object')

In [62]:
tabelas_df['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha'],
      dtype='object')

In [63]:
vendas_df=pd.DataFrame()

vendas_df=tabelas_df['TargetEstatistico']

In [64]:
vendas_df['AnoMes']=vendas_df['Data de Faturamento'].apply(lambda info: f'{str(info.year)}/{str(info.month)}')

In [65]:
vendas_df=vendas_df[['AnoMes','SKU','Qtde']].groupby(['AnoMes','SKU'],as_index=False).sum()

In [66]:
vendas_df=vendas_df.pivot(index='SKU',columns='AnoMes',values='Qtde').reset_index()

In [67]:
vendas_df

AnoMes,SKU,2018/7,2019/7,2020/7,2021/7,2022/7
0,3,1924.0,1630.0,569.0,1187.0,1042.0
1,8,179.0,NaN,NaN,NaN,NaN
2,12,2034.0,1781.0,867.0,1453.0,1916.0
3,34,2117.0,2043.0,621.0,1469.0,1388.2
4,35,318.0,369.0,138.0,242.0,274.0
...,...,...,...,...,...,...
620,9587,NaN,NaN,NaN,NaN,1.0
621,9588,NaN,NaN,NaN,NaN,1.0
622,9589,NaN,NaN,NaN,NaN,226.0
623,9590,NaN,NaN,NaN,NaN,291.0


In [68]:
tabelas_df['Produto']=tabelas_df['Produto'].loc[tabelas_df['Produto']['Status']=='ATIVO']

In [71]:
colunas=[l for l in vendas_df.columns.tolist() if l!='SKU']

valores=[]

for indice,linha in vendas_df.iterrows():
    
    soma=0
    
    contagem=0
    
    for col in colunas:
        
        vendas_df.loc[vendas_df[col].isnull(),col]=0
        
        soma+=linha[col].sum()
        
        if(linha[col]>0):
            
            contagem+=1
                        
            pass

        pass
    
    media=round(soma/contagem,2)
    
    valores.append(media)
       
    pass

vendas_df['Média']=valores

In [72]:
tabelas_df['Produto'].merge(vendas_df,on='SKU',how='inner')

,SKU,Cód. Fabricante,Produto,Status,Fabricante,Departamento,Seção,Categoria,Linha,2018/7,2019/7,2020/7,2021/7,2022/7,Média
0,3,101111036,POLPINHA DE ABACAXI (100G),ATIVO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,1924.0,1630.0,569.0,1187.0,1042.0,1270.40
1,8,101112606,POLPINHA DE ACAI (100G),ATIVO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,179.0,0.0,0.0,0.0,0.0,179.00
2,12,101110206,POLPINHA DE ACEROLA (100G),ATIVO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,2034.0,1781.0,867.0,1453.0,1916.0,1610.20
3,34,101122213,"IQF AMORA CONGELADOS (1,200 KG)",ATIVO,DE MARCHI,CONGELADOS,FRUTAS CONGELADAS,FRUTAS > 1KG,"FRUTA 1,2KG",2117.0,2043.0,621.0,1469.0,1388.2,1527.64
4,35,101112306,POLPINHA DE AMORA (100G),ATIVO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,318.0,369.0,138.0,242.0,274.0,268.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,9586,,"CONG. PICKERS CROQUETE SUINO MCCAIN 1,05KG",ATIVO,MCCAIN,CONGELADOS,BATATAS CONGELADAS,APERITIVOS,APERITIVOS,0.0,0.0,0.0,0.0,10.0,10.00
415,9587,2282,EMPADA 20G PALMITO CONGELADO DM 1KG,ATIVO,CUNHA,CONGELADOS,SALGADOS,EMPADA,SALGADOS,0.0,0.0,0.0,0.0,1.0,1.00
416,9588,7897424401253,EMPADA 20G FRANGO CONGELADO DM 1KG,ATIVO,CUNHA,CONGELADOS,SALGADOS,EMPADA,SALGADOS,0.0,0.0,0.0,0.0,1.0,1.00
417,9589,20.104.04,"CONG. BATATA MCCAIN SMILES NAC (1,05KG)",ATIVO,MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA ESPECIALIDADE,BATATA MASSA,0.0,0.0,0.0,0.0,226.0,226.00
